In [410]:
# This code base 'is' built to parse and extract specific values from RC1 and NRW documents (ie: South African documents)
# and the Japanese Export Certificate document. The Main technologies used are:
# - Python (Primary Language of the code)
# - Google cloud Vision (document scanner and computer vision)

In [424]:
# imports
import os, io
from google.cloud import vision
from google.cloud.vision_v1 import types
from google.cloud import translate_v2 as translate
import numpy as np
import json
import re
import sys
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from enum import Enum
from termcolor import colored
import uuid
from itertools import groupby
import statistics
# for matplotlib
%matplotlib inline 


os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'Demz_vision_API_token.json' # google API credentials

# Path to the rc1
rc1_1 = 'rc1-1.jpg'
rc1_2 = 'rc1-2.jpg'
FOLDER_PATH = 'C:\\Users\\USER\\Desktop\\NEW_DEMZ\\OCR\OCR_Works\\Google\\rc1\\images'

# # Path to the rc1
# nrw_1 = 'nrw-1.jpg'
# nrw_2 = 'nrw-2.jpg'
# nrw_3 = 'nrw-3.jpg'
# FOLDER_PATH = 'C:\\Users\\USER\\Desktop\\NEW_DEMZ\\OCR\OCR_Works\\Google\\nrw\\images'

japan1 = 'japanese.jpg'
# japan2 = 'jap_exp_cert_sample_1.jpg'
# japan3 = 'jap_exp_cert_sample_2.png'
# japan4 = 'jap_exp_cert_sample_3.jpg'
# japan5 = 'jap_exp_cert_sample_1_auto_x1_colored_toned.jpg'
FOLDER_PATH = 'C:\\Users\\USER\\Desktop\\NEW_DEMZ\\OCR\\OCR_Works\\Google\\japanese\\images'

image = japan1 
image = os.path.join(FOLDER_PATH, image) # full path to the image

# google vision and translation APIs
vision_client = vision.ImageAnnotatorClient()
translate_client = translate.Client()
client = vision.ImageAnnotatorClient()

In [435]:
def jap_parser(image):
    """This functionality is to parse the Japanese document. Feel free to add updates"""
    
    # Open and Read contents on the documents
    with io.open(image, 'rb') as image_file1:
            content = image_file1.read()
    content_image = vision.Image(content=content) # Reading the Image Content 
    rc1_response = vision_client.document_text_detection(
        image=content_image,
        image_context={"language_hints": 'ja'},  # Japanese
    )
    rc1_texts = rc1_response.text_annotations # Text Response

    # face detection
    response_face = client.face_detection(image=content_image)
    faceAnnotations = response_face.face_annotations
    
    
    # rearrange the Response using the bbox values
    items = []
    lines = {}

    for text in rc1_response.text_annotations[1:]:
        left_x_axis = text.bounding_poly.vertices[0].x # top left
        left_y_axis = text.bounding_poly.vertices[0].y # top left
        top_x_axis = text.bounding_poly.vertices[1].x # top
        top_y_axis = text.bounding_poly.vertices[1].y # top
        right_x_axis = text.bounding_poly.vertices[2].x # right
        right_y_axis = text.bounding_poly.vertices[2].y # right
        bottom_x_axis = text.bounding_poly.vertices[3].x # bottom
        bottom_y_axis = text.bounding_poly.vertices[3].y # bottom
        
        # Arranging the bounding_polys
        if left_y_axis not in lines:
            lines[left_y_axis] = [(left_y_axis, bottom_y_axis), []]
        for s_top_y_axis, s_item in lines.items():
            if left_y_axis < s_item[0][1]:
                face_vertices = (['({0},{1})'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
                                            #    Left            # Bottom    # Right        # vertices.      # Text       incase you still want to rearrange them in the future
                lines[s_top_y_axis][1].append(([left_x_axis, bottom_y_axis, top_x_axis, (face_vertices)], text.description))
                break
    for _, item in lines.items():
        if item[1]:
            words = sorted(item[1], key=lambda t: t[0])
            items.append((item[0], ' '.join([word for _, word in words]), words))
            
    def find_common_space(items):
        """Find common line spaces in the document"""
        spaces = []
        for i,k in enumerate(items):
            try:
                calculated_space = items[i][2][0][0][1] - items[i -1][2][0][0][1]
                spaces.append(calculated_space)
            except:
                ...

        common_space = abs(statistics.median(spaces))

        return common_space

    # Split the content based on horizontal space
    new_content = []
    most_common_space = find_common_space(items)
    print(most_common_space)

    len_content = len(items) / 5.5

    for i,k in enumerate(items):
        try:
            if abs(items[i][2][0][0][1] - items[i -1][2][0][0][1]) >= most_common_space -len_content:
                new_content.append([])
                new_content.append(items[i][2])
            else:
                new_content.append(items[i][2])
        except:
            new_content.append(items[i][2])


    # group the contents based on horizontal space. grouping the contents anywhere we find '[]'
    new_content = [list(l) for i, l in groupby(new_content, bool) if i]

    # join list that the length is greater than one. so that it is easier to loop through
    for i in range(len(new_content)):
        if len(new_content[i]) > 1:
            new_content[i] = sum(new_content[i], [])
            new_content[i] = [new_content[i]]
        else:
            new_content[i] = new_content[i]


    # find common width of white space on the document
    def find_common_diff(content):
        diffs = []
        for i,j in enumerate(new_content):
            for u,t in enumerate(new_content[i][0]):
                try:
                    difference = abs(new_content[i][0][u][0][2] - new_content[i][0][u-1][0][0])
                    diffs.append(difference)
                except:
                    ...
        common_diffs = abs(statistics.median(diffs))
        return common_diffs

    # find common width of white space on a line
    def find_common_diff_in_line(content):
        diffs = []
        for i,j in enumerate(new_content):
            temp_diffs = []
            for u,t in enumerate(new_content[i][0]):
                try:
                    difference = abs(new_content[i][0][u][0][0] - new_content[i][0][u-1][0][2])
                    temp_diffs.append(difference)
                except:
                    ...
            diffs.append(temp_diffs)
        return diffs


    def check_for_max_width(new_content):
        """Find the texts with the hightest width. The purpose of this that if a text if very long that it 
        takes up almost half line of the document then we will just use the most common width in the line instead of the document
        """
        widths = []
        for i,j in enumerate(new_content):
            temp_diffs = []
            for u,t in enumerate(new_content[i][0]):
                try:
                    text_width = abs(new_content[i][0][u][0][0] - new_content[i][0][u][0][2])
                    temp_diffs.append(text_width)
                except:
                    ...
            widths.append(temp_diffs)
        max_width = max(widths)
        return widths

    # split line content based on vertical spacing
    contents = []
    max_text_width = 600
    most_common_diffs = find_common_diff(new_content)

    for i,j in enumerate(new_content):
        """This snippet groups the document based on vertival spacing"""
        temp_content = []
        most_common_diffs_in_a_line = abs(statistics.median(find_common_diff_in_line(new_content)[i]))
        text_width = check_for_max_width(new_content)[i] # checking for max width of texts
        max_width = max(text_width)
        for u,t in enumerate(new_content[i][0]):
            try:
                difference = abs(new_content[i][0][u][0][0] - new_content[i][0][u-1][0][2])
                if most_common_diffs >= most_common_diffs_in_a_line and max_width < max_text_width:
                    if difference >= most_common_diffs_in_a_line+100: # getting the min width of white space in that line
                        temp_content.append([])
                        temp_content.append(new_content[i][0][u])

                    else:
                        temp_content.append(new_content[i][0][u]) 
                elif most_common_diffs >= most_common_diffs_in_a_line and max_width > max_text_width:

                    if difference > most_common_diffs_in_a_line-15: # getting the min width of white space in that line
                        temp_content.append([])
                        temp_content.append(new_content[i][0][u])

                    else:
                        temp_content.append(new_content[i][0][u])
                else:
    #                 This bias (20) should increase or decrease depend on document
                    if difference > most_common_diffs+20: # getting the most common width of white space in the doc
                        temp_content.append([])
                        temp_content.append(new_content[i][0][u])

                    else:
                        temp_content.append(new_content[i][0][u])
            except:
                ...
        # grouping the contents that are family :). grounping the contents anywhere we see '[]'
        new_temp_content = [list(l) for i, l in groupby(temp_content, bool) if i]
        contents.append(new_temp_content)    
   



    # Extract the first 3 contents that the starting position is less than 80 and the bottom position is less than 300 
    # This is specific to what is need to extract at the moment for the Japanese document. so we are not parsing the whole document
    max_bottom_position = 300
    max_start_pos = 80

    new_contents = []
    for item in range(len(contents)):
        context_in_line = contents[item][0]
        start_position = contents[item][0][0][0][0]
        bottom_position = contents[item][0][0][0][1]
        if start_position < max_start_pos and bottom_position < max_bottom_position:
            new_contents.append(context_in_line)
        else:
            ...


    # get the bound for croping out the detected text
    bounds = []
    for i in range(len(new_contents)):
        line_bound = []
        for u in range(len(new_contents[i]) - len(new_contents[i])+1):
            first_top = new_contents[i][u][0][3][0]
            last_bottom = new_contents[i][u-1][0][3][2]
            bound = [first_top, last_bottom]
            line_bound.append(bound)
        bounds.append(line_bound)

    # Get crop hint
    crop_hints_params = vision.CropHintsParams(aspect_ratios=[1.77])
    def get_crop_hint(crop_hints):
        """Detect crop hints on a single image and return the first result."""
        with io.open(crop_hints, 'rb') as image_file1:
            content = image_file1.read()

        # content = crop_hints
        image = vision.Image(content=content)

        image_context = vision.ImageContext(crop_hints_params=crop_hints_params)

        response = client.crop_hints(image=image, image_context=image_context)
        hints = response.crop_hints_annotation.crop_hints

        # Get bounds for the first crop hint using an aspect ratio of 1.77.
        vertices = hints[0].bounding_poly.vertices

        return vertices

    def crop_Text_hint(image_file, TextAnnotations):
        """Crop the image using the hints in the vector list."""
        vects = get_crop_hint(image_file)
        TextAnnotations = TextAnnotations
        try:
            images = []
            for i in range(len(TextAnnotations)):
                bound_1 = TextAnnotations[i][0][0]
                bound_2 = TextAnnotations[i][0][1]
                bound_1 = eval(bound_1)
                bound_2 = eval(bound_2)

                im = Image.open(image_file)

                im2 = im.crop([bound_1[0], bound_1[1],
                              bound_2[0] - 1, bound_2[1] - 1])
    #             unique_id = uuid.uuid4().hex
    #             output = f'output_{unique_id}.jpg'
    #             im2.save(output, 'JPEG')
                images.append(im2)
            return images
        except Exception as e:
            print(e)
            print("No Image Detected")

    croped_out = crop_Text_hint(os.path.join(FOLDER_PATH, image), bounds)
    # face

    max_diff = 10
    main_contents = []
    for i,j in enumerate(new_contents):
        """ Grouping the contents based on vertical spacing . This is where we get the get the keys and values"""
        temp_content = []
        for u,t in enumerate(new_contents[i]):
            try:
                print(new_contents[i][u][0][0])
                difference = abs(new_contents[i][u][0][0] - new_contents[i][u-1][0][2])
                print(difference)
                if difference > max_diff: # getting the most common width of white space in the doc
                    temp_content.append([])
                    temp_content.append(new_contents[i][u])
                else:
                    temp_content.append(new_contents[i][u])
            except:
                temp_content.append(new_contents[i][u])
        new_temp_content = [list(l) for i, l in groupby(temp_content, bool) if i]
        main_contents.append(new_temp_content)
    # print(main_contents)

    jap_contents = []
    images_of_extracted_values = []
    max_starting_pos = 100

    for i in range(len(main_contents)):
        """Convert the lists we grouped above into keys and values"""
        keys = ''
        values = ''
        for u in range(len(main_contents[i])):
            left_bbox = main_contents[i][u][0][0][0]
            if left_bbox <=max_starting_pos:
                try:
                    for j in range(len(main_contents[i][u])):
                        text = main_contents[i][u][j][1]
                        keys += ' '+text
                except:
                    ...
            else:
                try:
                    for j in range(len(main_contents[i][u])):
                        text = main_contents[i][u][j][1]
                        if abs(main_contents[i][u][j][0][0] - main_contents[i][u-1][j][0][0]) <= 41:
                            values += ''+text
                        else:
                            values += ' '+text
                except:
                    ...
        # Translate the keys
        text_translate = translate_client.translate(keys, target_language='en')['translatedText']
        keys = keys + " / " +  text_translate
        jap_contents.append({keys:values}) # Keys and values
        ith_image = croped_out[i]
        try: # convert the image to bytes to enable html display
#             buf = io.BytesIO()
#             ith_image.save(buf, format='PNG')
#             ith_image = buf.getvalue()
            images_of_extracted_values.append(ith_image)
        except:
            images_of_extracted_values.append(croped_out[i])

    return jap_contents, images_of_extracted_values # return the contents and images of the contents

In [436]:
 # Find the most common space
def af_parser(image):
    """This functionality is built to parse the South African documents"""
    client = vision.ImageAnnotatorClient()

    # Open and Read contents on the documents
    with io.open(image, 'rb') as image_file1:
            content = image_file1.read()
    content_image = vision.Image(content=content) # Reading the Image Content
    rc1_response = client.text_detection(image=content_image) # Text Detection 
    rc1_texts = rc1_response.text_annotations # Text Response

    # face detection
    response_face = client.face_detection(image=content_image)
    faceAnnotations = response_face.face_annotations
    
    
    # rearrange the Response using the bbox values
    items = []
    lines = {}

    for text in rc1_response.text_annotations[1:]:
        left_x_axis = text.bounding_poly.vertices[0].x # top left
        left_y_axis = text.bounding_poly.vertices[0].y # top left
        top_x_axis = text.bounding_poly.vertices[1].x # top
        top_y_axis = text.bounding_poly.vertices[1].y # top
        right_x_axis = text.bounding_poly.vertices[2].x # right
        right_y_axis = text.bounding_poly.vertices[2].y # right
        bottom_x_axis = text.bounding_poly.vertices[3].x # bottom
        bottom_y_axis = text.bounding_poly.vertices[3].y # bottom
        
        # Arranging the bounding_polys
        if left_y_axis not in lines:
            lines[left_y_axis] = [(left_y_axis, bottom_y_axis), []]
        for s_top_y_axis, s_item in lines.items():
            if left_y_axis < s_item[0][1]:
                face_vertices = (['({0},{1})'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
                                           #     Left          # Bottom      # top Right        # vertices.        # Text            incase you still want to rearrange them in the future
                lines[s_top_y_axis][1].append(([left_x_axis, bottom_y_axis, top_x_axis, (face_vertices)], text.description))
                break
    for _, item in lines.items():
        if item[1]:
            words = sorted(item[1], key=lambda t: t[0])
            items.append((item[0], ' '.join([word for _, word in words]), words))
            
    def find_common_space(items):
        """Find common line spaces in the document"""
        spaces = []
        for i,k in enumerate(items):
            try:
                calculated_space = items[i][2][0][0][1] - items[i -1][2][0][0][1]
                spaces.append(calculated_space)
            except:
                ...

        common_space = abs(statistics.median(spaces))

        return common_space

    # Split the content based on horizontal space

    new_content = []

    most_common_space = find_common_space(items)

    len_content = len(items) / 5.5
    start_pos = 300

    for i,k in enumerate(items):
        prev_item = items[i-1][2][0]
        left_bbox = items[i][2][0][0][0]
        
        prev_left_bbox = items[i-1][2][0][0][0]
        curr_first_letter = items[i][2][0][1][0]
        prev_first_letter = items[i-1][2][0][1][0]
        #Check if a line has keys and values by checking the lenght
        Curr_item_last_left_bbox = []
        for u in range(len(items[i])):
            try:
                if items[i][2][u][0][0] < 790:
                    Curr_item_last_left_bbox.append(items[i][2][u][0][0])
            except:
                pass
        try:
#             if abs(items[i][2][0][0][1] - items[i -1][2][0][0][1]) >= most_common_space -len_content and not (prev_first_letter.isupper() and curr_first_letter.islower() and len(items[i][2]) == 1) and not (left_bbox > start_pos and not len(prev_item) == 1):
            if abs(items[i][2][0][0][1] - items[i -1][2][0][0][1]) >= most_common_space -len_content and not (prev_first_letter.isupper() and curr_first_letter.islower() and len(Curr_item_last_left_bbox) == 1) and not (left_bbox > start_pos and not len(prev_item) == 1):
                new_content.append([])
                new_content.append(items[i][2])
            else:
                new_content.append(items[i][2])
        except:
            new_content.append(items[i][2])


    # group the contents based on horizontal space. grouping the contents anywhere we find '[]'
    new_content = [list(l) for i, l in groupby(new_content, bool) if i]

    # join list that the length is greater than one
    for i in range(len(new_content)):
        if len(new_content[i]) > 1:
            new_content[i] = sum(new_content[i], [])
            new_content[i] = [new_content[i]]
        else:
            new_content[i] = new_content[i]

    # find common width of white space on the document
    def find_common_diff(content):
        diffs = []
        for i,j in enumerate(new_content):
            for u,t in enumerate(new_content[i][0]):
                try:
                    difference = abs(new_content[i][0][u][0][2] - new_content[i][0][u-1][0][0])
                    diffs.append(difference)
                except:
                    ...
        common_diffs = abs(statistics.median(diffs))
        return common_diffs

    # find common width of white space on a line
    def find_common_diff_in_line(content):
        diffs = []
        for i,j in enumerate(new_content):
            temp_diffs = []
            for u,t in enumerate(new_content[i][0]):
                try:
                    difference = abs(new_content[i][0][u][0][0] - new_content[i][0][u-1][0][2])
                    temp_diffs.append(difference)
                except:
                    ...
            diffs.append(temp_diffs)
        return diffs

    def check_for_max_width(new_content):
        '''Check for max with of a text in a line'''
        """Find the texts with the hightest width. The purpose of this that if a text if very long that it 
        takes up almost half line of the document then we will just use the most common width in the line instead of the document
        """
        widths = []
        for i,j in enumerate(new_content):
            temp_diffs = []
            for u,t in enumerate(new_content[i][0]):
                try:
                    text_width = abs(new_content[i][0][u][0][0] - new_content[i][0][u][0][2])
                    temp_diffs.append(text_width)
                except:
                    ...
            widths.append(temp_diffs)
        max_width = max(widths)
        return widths

    # split line content based on vertical spacing
    contents = []
    max_text_width = 600
    most_common_diffs = find_common_diff(new_content)

    for i,j in enumerate(new_content):
        """This snippet groups the document based on vertival spacing. First move to getting the Keys and values"""
        temp_content = []
        most_common_diffs_in_a_line = abs(statistics.median(find_common_diff_in_line(new_content)[i]))
        text_width = check_for_max_width(new_content)[i]
        max_width = max(text_width)
        for u,t in enumerate(new_content[i][0]):
            try:
                difference = abs(new_content[i][0][u][0][0] - new_content[i][0][u-1][0][2])
                if most_common_diffs >= most_common_diffs_in_a_line and max_width < max_text_width:
                    if difference >= most_common_diffs_in_a_line+20: # getting the min width of white space in that line
                        temp_content.append([])
                        temp_content.append(new_content[i][0][u])

                    else:
                        temp_content.append(new_content[i][0][u]) 
                elif most_common_diffs >= most_common_diffs_in_a_line and max_width > max_text_width:
                    if difference > most_common_diffs_in_a_line-15: # getting the min width of white space in that line
                        temp_content.append([])
                        temp_content.append(new_content[i][0][u])

                    else:
                        temp_content.append(new_content[i][0][u])
                else:
                    if difference > most_common_diffs-65: # getting the most common width of white space in the doc
                        temp_content.append([])
                        temp_content.append(new_content[i][0][u])

                    else:
                        temp_content.append(new_content[i][0][u])
            except:
                ...
        # Grouping the contents anywhere we find '[]'
        new_temp_content = [list(l) for i, l in groupby(temp_content, bool) if i]
        contents.append(new_temp_content)    

    def extract_keys_and_values(contents):
        """ This functionality extracts they keys and values as in the document structure"""
        def check_for_non_english(new_content):
            """this function was built to Counts the number of non-english and english texts in a line. but because of the 
            time complexity the algorithm was removed while I still keep the function name. incase if you're wondering
            So what this function does is to counts sentences/words that the starting position is less than 780 and the give the line a score"""
            max_pos = 780 # this is the max starting position that an english word does exceed
            translate_client = translate.Client()
            target = 'en'
            detected_words = []
            for i,j in enumerate(new_content):
                temp_words = []
                for u,t in enumerate(new_content[i]):
                    counts = {'en':0, 'af':0} # score counts. "en" is english and "af" is any other language
                    left_bbox = contents[i][u][0][0][0]
                    for s,r in enumerate(new_content[i][u]):
                        try:
                            text = new_content[i][u][s][1]

                            if left_bbox < max_pos:
                                counts['en'] += 1
                            else:
                                counts['af'] += 1
                        except:
                            ...
                    temp_words.append(counts)
                detected_words.append(temp_words)
            return detected_words

        word_detection = check_for_non_english(contents)

        new_contents = []
        max_position = 1200
        list_of_special_keywords = ['RC1', 'RNC', 'RTS', 'NRW'] # special key words to add to the extracted values

        for i,j in enumerate(contents): # 
            temp_content = []
            word_detected = word_detection[i]
            content = contents[i]
            for u,t in enumerate(contents[i]):
                left_bbox = contents[i][u][0][0][0]
                if word_detected[u]['en'] > word_detected[u]['af'] and left_bbox < max_position or contents[i][u][0][1] in list_of_special_keywords:
                    temp_content.append(content[u])
            new_contents.append(temp_content)

        # get the left bbox of the keys
        new_contents = [x for x in new_contents if x != []]
        return new_contents

    def crop_extracted_info(new_contents):
        # get the bound for croping out the detected text
        bounds = []
        for i in range(len(new_contents)):
            line_bound = []
            if len(new_contents[i]) > 1:
                for u in range(len(new_contents[i])):
                    left_max = ()
                    top_max = ()
                    right_max = ()
                    bottom_max = ()
                    try:
                        top = []
                        bottom = []
                        right = []
                        left = []
                        right_elements_list = []
                        left_elements_list = []
                        # Separate the bbox accordingly 
                        if new_contents[i][u][0][0][0] > 400:
                            right_elements_list.append(new_contents[i][u])
                        else:
                            left_elements_list.append(new_contents[i][u])
                        for o in range(len(right_elements_list)):
                            if right_elements_list[o] != []: # get the bbox accordingly for croping the extracted text
                                for e in range(len(right_elements_list[o])):
                                    left.append(eval(right_elements_list[o][e][0][3][0]))
                                    top.append(eval(right_elements_list[o][e][0][3][1]))
                                    right.append(eval(right_elements_list[o][e-1][0][3][2])) # the last items from the right
                                    bottom.append(eval(right_elements_list[o][e][0][3][3]))
                        for o in range(len(left_elements_list)):
                            if left_elements_list[o] != []: # get the bbox accordingly for croping the extracted text
                                for e in range(len(left_elements_list[o])):
                                    left.append(eval(left_elements_list[o][e][0][3][0]))
                                    top.append(eval(left_elements_list[o][e][0][3][1]))
                                    right.append(eval(left_elements_list[o][e][0][3][2]))
                                    bottom.append(eval(left_elements_list[o][e][0][3][3]))

                        # get the max and min bounds for top, left ... etc

                        left_max = sorted(left,key=lambda x: x[0], reverse=False)[0]
                        top_max = sorted(top,key=lambda x: x[0], reverse=True)[0]
                        right_max = sorted(right,key=lambda x: x[0], reverse=True)[0]
                        bottom_max = sorted(bottom,key=lambda x: x[0], reverse=True)[0]

                    except:
                        ...

                    bound = [left_max, top_max, right_max, bottom_max]
                    line_bound.append(bound)
                bounds.append(line_bound)
            else:
                for u in range(len(new_contents[i])):
                    first_top = []
                    last_bottom = []
                    try:
                        first_top = new_contents[i][u][0][0][3][0]
                        last_bottom = new_contents[i][u-1][-1][0][3][2]
                    except:
                        ...

                    bound = [first_top, last_bottom]
                    line_bound.append(bound)
                bounds.append(line_bound)

        # Get crop hint
        crop_hints_params = vision.CropHintsParams(aspect_ratios=[1.77])
        def get_crop_hint(crop_hints):
            """Detect crop hints on a single image and return the first result."""
            with io.open(crop_hints, 'rb') as image_file1:
                content = image_file1.read()

            # content = crop_hints
            image = vision.Image(content=content)
            image_context = vision.ImageContext(crop_hints_params=crop_hints_params)

            response = client.crop_hints(image=image, image_context=image_context)
            hints = response.crop_hints_annotation.crop_hints

            # Get bounds for the first crop hint using an aspect ratio of 1.77.
            vertices = hints[0].bounding_poly.vertices

            return vertices

        def crop_Text_hint(image_file, TextAnnotations):
            """Crop the image using the hints in the vector list."""
            vects = get_crop_hint(image_file)
            TextAnnotations = TextAnnotations
            try:
                images = []
                for i in range(len(TextAnnotations)):
                    try:
                        sum_bounds = []
                        for c in range(len(TextAnnotations[i])):
                            sum_bounds.append(TextAnnotations[i][c])
                        sum_bounds = sum(sum_bounds, [])
                        bound_1 = TextAnnotations[i][0][0]
                        bound_2 = TextAnnotations[i][0][1]
                        bound_3 = sorted(sum_bounds,key=lambda x: x[0], reverse=True)[0]
                        bound_4 = TextAnnotations[i][-1][3]
                        bound_1 = bound_1 # left
                        bound_2 = bound_2 # top
                        bound_3 = bound_3 # right
                        bound_4 = bound_4 # bottom
                        im = Image.open(image_file)

                        # check which of the values are higher
                        if bound_3[0] > bound_3[1]:

                            im2 = im.crop([bound_1[0]-1, bound_2[1]-7,
                                          bound_3[0]+5, bound_4[1] + 5]) # added some padding
                            images.append(im2)
                        else:
                            im2 = im.crop([bound_1[0]-1, bound_2[1]-10,
                                      bound_3[0]+5, bound_4[1]+5]) # added some padding
                            images.append(im2)
                    except:
                        bound_1 = TextAnnotations[i][0][0]
                        bound_2 = TextAnnotations[i][0][1]
                        # sum the list if the len is greater than 1
                        try:
                            bound_1 = sum(bound_1[0], [])
                            bound_2 = sum(bound_2[0], [])
                        except:
                            ...
                        try:
                            bound_1 = eval(bound_1)
                            bound_2 = eval(bound_2)
                        except:
                            ...

                        im = Image.open(image_file)

                        im2 = im.crop([bound_1[0]-1, bound_1[1]-7,
                                      bound_2[0] + 5, bound_2[1] + 5]) # added some padding
                        images.append(im2)
                return images
            except Exception as e:
                exc_type, exc_obj, exc_tb = sys.exc_info()
                fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
                # print(exc_type, fname, exc_tb.tb_lineno)
                # print(e)
                # print("No Image Detected")

        croped_out = crop_Text_hint(image, bounds)

        return croped_out

    def get_values(new_contents):
        """This functionality get the keys and values in a key pair or dictionary"""
        left_bbox_lists = []
        for i in range(len(new_contents)):
            try:
                left_bbox = new_contents[i][0][0][0][0]
                left_bbox_lists.append(left_bbox)
            except:
                left_bbox = 0
                left_bbox_lists.append(left_bbox)

        max_left_bbox = max(left_bbox_lists)

        # getting the keys and values 
        rc1_contents = []
        max_starting_pos = 250

        for i in range(len(new_contents)):
            keys = ''
            values = ''
            for u in range(len(new_contents[i])):
                left_bbox = new_contents[i][u][0][0][0]
                if left_bbox <= max_left_bbox and left_bbox <=max_starting_pos:
                    try:
                        for j in range(len(new_contents[i][u])):
                            text = new_contents[i][u][j][1]
                            keys += ' '+text
                    except:
                        ...
                else:
                    try:
                        for j in range(len(new_contents[i][u])):
                            text = new_contents[i][u][j][1]
                            values += ' '+text
                    except:
                        ...

            rc1_contents.append({keys:values})
        return rc1_contents

#     contents = arrange_the_texts(rc1_response)

    contents = extract_keys_and_values(contents)

    # crop out the values to be extracted
    croped_out = crop_extracted_info(contents)

    # extract the values
    contents = get_values(contents)
    # keys needed to be pulled out from the document
    keys_to_pull_out_RNC = ['Registering authority', 'Traffic register number', 'Name', 'Postal address', 'Street address', 'Address where notices must be served', 'Control number', 'Issue number', 'Date of issue']
    keys_not_to_pull_out_RC1 = [ "4024", "at Registering which registered authority", "RET(7)(2005/02)", "Republic of South Africa" ]
    keys_not_to_pull_out_NRW = ["NRW(2)(2003/10)", "NOTICE", "PARTICULARS (National Road Traffic OF Act, VEHICLE"]

    # change the keys to upper case
    keys_to_pull_out_RNC = [i.upper().strip() for i in keys_to_pull_out_RNC]
    keys_not_to_pull_out_RC1 = [i.upper().strip() for i in keys_not_to_pull_out_RC1]
    keys_not_to_pull_out_NRW = [i.upper().strip() for i in keys_not_to_pull_out_NRW]

    extracted_values = []
    images_of_extracted_values = []

    # check if the special keys are in the document, if so pull the right keys
    for e,f in contents[0].items():
        if 'RNC' in f:
            for i in range(len(contents)):
                for key, value in contents[i].items():
                    key = ' '.join(key.split())
                    if key.upper() in keys_to_pull_out_RNC:
                        extracted_values.append(contents[i])
                        ith_image = croped_out[i]
                        try: # convert the image to bytes to enable html display
    #                             buf = io.BytesIO()
    #                             ith_image.save(buf, format='PNG')
    #                             ith_image = buf.getvalue()
                            images_of_extracted_values.append(ith_image)
                        except:
                            images_of_extracted_values.append(croped_out[i]) # this is from the images croped out
        elif 'RC1' in f:
            for i in range(len(contents)):
                for key, value in contents[i].items():
                    key = ' '.join(key.split())
                    if key.upper() in keys_not_to_pull_out_RC1:
                        ...
                    else:
                        if key != '':
                            extracted_values.append(contents[i])
                            ith_image = croped_out[i]
                            try: # convert the image to bytes to enable html display
    #                                 buf = io.BytesIO()
    #                                 ith_image.save(buf, format='PNG')
    #                                 ith_image = buf.getvalue()
                                images_of_extracted_values.append(ith_image)
                            except:
                                images_of_extracted_values.append(croped_out[i]) # this is from the images croped out
        elif 'NRW' in f:
            for i in range(len(contents)):
                for key, value in contents[i].items():
                    key = ' '.join(key.split())
                    if key.upper() in keys_not_to_pull_out_NRW:
                        ...
                    else:
                        if key != '':
                            extracted_values.append(contents[i])
                            ith_image = croped_out[i]
                            try: # convert the image to bytes to enable html display
    #                                 buf = io.BytesIO()
    #                                 ith_image.save(buf, format='PNG')
    #                                 ith_image = buf.getvalue()
                                images_of_extracted_values.append(ith_image)
                            except:
                                images_of_extracted_values.append(croped_out[i]) # this is from the images croped out
        else:
            extracted_values.append({'Oops!':'document has a different structure'})
    print(extracted_values)
    return extracted_values, images_of_extracted_values

In [437]:
def parser(image):
    """This functionality does a check to see if the loaded document is the japanese document or the south african. And then
    calls the apropriate algorithm"""
    is_japanese_parser = False
    
    client = vision.ImageAnnotatorClient()

    # Open and Read contents on the documents
    with io.open(image, 'rb') as image_file1:
            content = image_file1.read()
    content_image = vision.Image(content=content) # Reading the Image Content
    rc1_response = client.document_text_detection(image=content_image) # Text Detection 
    rc1_texts = rc1_response.text_annotations # Text Response
#     print(rc1_texts)

    # face detection
    response_face = client.face_detection(image=content_image)
    faceAnnotations = response_face.face_annotations
    
    
    # rearrange the Response using the bbox values
    items = []
    lines = {}

    for text in rc1_response.text_annotations[1:]:
        left_x_axis = text.bounding_poly.vertices[0].x # top left
        left_y_axis = text.bounding_poly.vertices[0].y # top left
        top_x_axis = text.bounding_poly.vertices[1].x # top
        top_y_axis = text.bounding_poly.vertices[1].y # top
        right_x_axis = text.bounding_poly.vertices[2].x # right
        right_y_axis = text.bounding_poly.vertices[2].y # right
        bottom_x_axis = text.bounding_poly.vertices[3].x # bottom
        bottom_y_axis = text.bounding_poly.vertices[3].y # bottom

        if left_y_axis not in lines:
            lines[left_y_axis] = [(left_y_axis, bottom_y_axis), []]
        for s_top_y_axis, s_item in lines.items():
            if left_y_axis < s_item[0][1]:
                face_vertices = (['({0},{1})'.format(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices])
                lines[s_top_y_axis][1].append(([left_x_axis, bottom_y_axis, top_x_axis, (face_vertices)], text.description))
                break
    for _, item in lines.items():
        if item[1]:
            words = sorted(item[1], key=lambda t: t[0])
            items.append((item[0], ' '.join([word for _, word in words]), words))

    # Check if the document is the japanese document
    if items[0][2][0][1][0] == '番':
        is_japanese_parser = True
            
        
    if is_japanese_parser:
        ex_vals, img_of_ex_vals = jap_parser(image) # Japanese document Algorithm
        return ex_vals, img_of_ex_vals
    else:
        ex_vals, img_of_ex_vals = af_parser(image) # South African document Algorithm
        return ex_vals, img_of_ex_vals
        

In [438]:
ex_vals, img_of_ex_vals = parser(image)

33
71
235
217
45
70
515
172
7
281
20
78
424
189
48
223
20
261
25
387
120
422
20
455
20
495
28


In [439]:
ex_vals

[{' 番号 / number': ' 00858'},
 {' 整理 番号 / Reference number': ' 3911024991745631'},
 {' 福岡 / Fukuoka': ' 499 4563'}]

In [440]:
ex_vals[0]

{' 番号 / number': ' 00858'}

In [441]:
img_of_ex_vals[0]